In [ ]:
#video processing
!pip install opencv-python
#training model
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install yolov5/requirements.txt
#label images
!pip install labelImg
!pip install setuptools
#getting screenshots
!pip install mss
#allows cpu to make movement 
!pip install pyautogui
!pip install keyboard

In [ ]:
#import statements
import cv2
import os
import re
import numpy as np
import shutil
import random
import os
from mss import mss
import torch
import pytesseract
from interception import beziercurve
import interception

In [ ]:
video_path = r".\Video"
temp_dir = r".\temp_frames"
train_dir = r".\dataset\images\train"
test_dir = r".\dataset\images\test"
test_split = 0.2  

for directory in [temp_dir, train_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)

#extracting frames from video and splitting them into training and validation sets
def process_video_directory(input_dir):
    video_extensions = ('.mp4') 
    video_files = [
        os.path.join(input_dir, f)
        for f in os.listdir(input_dir)
        if f.lower().endswith(video_extensions)
    ]

    if not video_files:
        print(f"No video files found in directory: {input_dir}")
        return

    print(f"Found {len(video_files)} videos to process.")
    extracted_frames = []

    for idx, video_path in enumerate(video_files):
        video_name = os.path.basename(video_path)
        print(f"\nProcessing: {video_name}...")
        count = extract_frames_from_video(video_path, video_name, extracted_frames)
        print(f"Extracted {count} frames.")

    random.shuffle(extracted_frames)
    split_idx = int(len(extracted_frames) * (1 - test_split))
    train_frames = extracted_frames[:split_idx]
    test_frames = extracted_frames[split_idx:]

    print(f"Splitting into {len(train_frames)} training and {len(test_frames)} test frames.")

    for frame in train_frames:
        shutil.move(os.path.join(temp_dir, frame), os.path.join(train_dir, frame))

    for frame in test_frames:
        shutil.move(os.path.join(temp_dir, frame), os.path.join(test_dir, frame))

    shutil.rmtree(temp_dir)
    print("Done.")

def extract_frames_from_video(video_path, video_name, frames):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_name}")
        return 0

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % int(fps) == 0:
            frame_filename = f"{os.path.splitext(video_name)[0]}_frame{frame_count}.jpg"
            frame_path = os.path.join(temp_dir, frame_filename)
            cv2.imwrite(frame_path, frame)
            frames.append(frame_filename)
            saved_count += 1

        frame_count += 1

    cap.release()
    return saved_count

process_video_directory(video_path)


Found 1 videos to process.

Processing: vid1.mp4...
Extracted 130 frames.
Splitting into 104 training and 26 test frames.
Done.


In [ ]:


#cleaning up the labels
def clean_label_filenames(root_folder):
    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".txt") and "-" in filename:
                new_name = filename.split("-", 1)[1]
                src_path = os.path.join(dirpath, filename)
                dst_path = os.path.join(dirpath, new_name)

                # Rename only if new name is different
                if src_path != dst_path:
                    os.rename(src_path, dst_path)
                else:
                    print(filename)
    print("done renaming")

clean_label_filenames("dataset\labels")


<>:20: SyntaxWarning: invalid escape sequence '\l'
<>:20: SyntaxWarning: invalid escape sequence '\l'
C:\Users\limju\AppData\Local\Temp\ipykernel_27788\3313658992.py:20: SyntaxWarning: invalid escape sequence '\l'
  clean_label_filenames("dataset\labels")


done renaming


In [ ]:
#training yolo model
!python yolov5/train.py --img 640 --batch 16 --epochs 50 --data dataset.yaml --weights yolov5s.pt

In [ ]:
#base for detection
model = torch.hub.load("ultralytics/yolov5", "custom", path="yolov5/runs/train/exp/weights/best.pt")

monitor = {"top": 0, "left": 0, "width": 1920, "height": 1080}

frame_count = 0
inference_interval = 5  

with mss() as sct:
    while True:
        frame = np.array(sct.grab(monitor))

        frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

        frame = cv2.resize(frame, (640, 480))

        # debugging (ensure that the bots are properly lablaed)
        if frame_count % inference_interval == 0:
            results = model(frame)
            detections = results.xyxy[0].cpu().numpy()  
        
            
            for detection in detections:
                x_min, y_min, x_max, y_max, confidence, class_id = detection
                label = f"Bot {confidence:.2f}"
                cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
                cv2.putText(frame, label, (int(x_min), int(y_min) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.imshow("Bot Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

        frame_count += 1

cv2.destroyAllWindows()

In [ ]:
# detect the scores using OCR
image = cv2.imread("dataset/images/train/vid1_frame1080.jpg")

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

x1_rel, y1_rel = 0.31, 0.055
x2_rel, y2_rel = 0.425, 0.1

x1 = int(x1_rel * width)
y1 = int(y1_rel * height)
x2 = int(x2_rel * width)
y2 = int(y2_rel * height)

score_roi = gray[y1:y2, x1:x2]


x1_rel, y1_rel = 0.56, 0.055
x2_rel, y2_rel = 0.6, 0.1

x1 = int(x1_rel * width)
y1 = int(y1_rel * height)
x2 = int(x2_rel * width)
y2 = int(y2_rel * height)

accuracy_roi = gray[y1:y2, x1:x2]


score_roi = cv2.threshold(score_roi, 127, 255, cv2.THRESH_BINARY_INV)[1]
accuracy_roi = cv2.threshold(accuracy_roi, 127, 255, cv2.THRESH_BINARY_INV)[1]

score_text = pytesseract.image_to_string(score_roi, config='--psm 7')
accuracy_text = pytesseract.image_to_string(accuracy_roi, config='--psm 7')

print("Score:", score_text.strip())
print("Accuracy:", accuracy_text.strip())


Score: 0
Accuracy: 29%


In [ ]:
# able to simualte accurate hand movement
# unable to use relative mouse movement
params = beziercurve.BezierCurveParams()
params.duration = 1.5        
params.smoothness = 0.005   

interception.move_relative(0, 10)